# Hospital Readmissions Data Analysis and Recommendations for Reduction

### Background
In October 2012, the US government's Center for Medicare and Medicaid Services (CMS) began reducing Medicare payments for Inpatient Prospective Payment System hospitals with excess readmissions. Excess readmissions are measured by a ratio, by dividing a hospital’s number of “predicted” 30-day readmissions for heart attack, heart failure, and pneumonia by the number that would be “expected,” based on an average hospital with similar patients. A ratio greater than 1 indicates excess readmissions.

### Exercise Directions

In this exercise, you will:
+ critique a preliminary analysis of readmissions data and recommendations (provided below) for reducing the readmissions rate
+ construct a statistically sound analysis and make recommendations of your own 

More instructions provided below. Include your work **in this notebook and submit to your Github account**. 

### Resources
+ Data source: https://data.medicare.gov/Hospital-Compare/Hospital-Readmission-Reduction/9n3s-kdb3
+ More information: http://www.cms.gov/Medicare/medicare-fee-for-service-payment/acuteinpatientPPS/readmissions-reduction-program.html
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bokeh.plotting as bkp
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from scipy import stats 

sns.set()

In [ ]:
# read in readmissions data provided
hospital_read_df = pd.read_csv('data/cms_hospital_readmissions.csv')

****
## Preliminary Analysis

In [ ]:
# deal with missing and inconvenient portions of data 
clean_hospital_read_df = hospital_read_df[hospital_read_df['Number of Discharges'] != 'Not Available'].copy()
clean_hospital_read_df.loc[:, 'Number of Discharges'] = clean_hospital_read_df['Number of Discharges'].astype(int)
clean_hospital_read_df = clean_hospital_read_df.sort_values('Number of Discharges')

In [ ]:
# generate a scatterplot for number of discharges vs. excess rate of readmissions
# lists work better with matplotlib scatterplot function
x = [a for a in clean_hospital_read_df['Number of Discharges'][81:-3]]
y = list(clean_hospital_read_df['Excess Readmission Ratio'][81:-3])

fig, ax = plt.subplots(figsize=(8,5))
ax.scatter(x, y,alpha=0.2)

ax.fill_between([0,350], 1.15, 2, facecolor='red', alpha = .15, interpolate=True)
ax.fill_between([800,2500], .5, .95, facecolor='green', alpha = .15, interpolate=True)

ax.set_xlim([0, max(x)])
ax.set_xlabel('Number of discharges', fontsize=12)
ax.set_ylabel('Excess rate of readmissions', fontsize=12)
ax.set_title('Scatterplot of number of discharges vs. excess rate of readmissions', fontsize=14)

ax.grid(True)
fig.tight_layout()

****

## Preliminary Report

Read the following results/report. While you are reading it, think about if the conclusions are correct, incorrect, misleading or unfounded. Think about what you would change or what additional analyses you would perform.

**A. Initial observations based on the plot above**
+ Overall, rate of readmissions is trending down with increasing number of discharges
+ With lower number of discharges, there is a greater incidence of excess rate of readmissions (area shaded red)
+ With higher number of discharges, there is a greater incidence of lower rates of readmissions (area shaded green) 

**B. Statistics**
+ In hospitals/facilities with number of discharges < 100, mean excess readmission rate is 1.023 and 63% have excess readmission rate greater than 1 
+ In hospitals/facilities with number of discharges > 1000, mean excess readmission rate is 0.978 and 44% have excess readmission rate greater than 1 

**C. Conclusions**
+ There is a significant correlation between hospital capacity (number of discharges) and readmission rates. 
+ Smaller hospitals/facilities may be lacking necessary resources to ensure quality care and prevent complications that lead to readmissions.

**D. Regulatory policy recommendations**
+ Hospitals/facilties with small capacity (< 300) should be required to demonstrate upgraded resource allocation for quality care to continue operation.
+ Directives and incentives should be provided for consolidation of hospitals and facilities to have a smaller number of them with higher capacity and number of discharges.

****
### Exercise

Include your work on the following **in this notebook and submit to your Github account**. 

A. Do you agree with the above analysis and recommendations? Why or why not?
   
B. Provide support for your arguments and your own recommendations with a statistically sound analysis:

   1. Setup an appropriate hypothesis test.
   2. Compute and report the observed significance value (or p-value).
   3. Report statistical significance for $\alpha$ = .01. 
   4. Discuss statistical significance and practical significance. Do they differ here? How does this change your recommendation to the client?
   5. Look at the scatterplot above. 
      - What are the advantages and disadvantages of using this plot to convey information?
      - Construct another plot that conveys the same information in a more direct manner.



You can compose in notebook cells using Markdown: 
+ In the control panel at the top, choose Cell > Cell Type > Markdown
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [ ]:
# Your turn

### Initial thought 

In the given dataset, there are multiple rows for same hospital/provider number. I strongly feel that whole analysis should be carried out after combining data for each hospital. To elaborate, each provider number should occur once in the dataset. The Predicted Readmission Rate and Expected Readmission Rate should be added for each provider number. And then Excess Readmission Ratio needs to be recalculated by dividing Predicted Readmission Rate with Expected Readmission Rate.

In the absence of above, the rest of the analysis will be flawed. 

Nevertheless, for this project as further cleaning is not required, I will work on the data as given.

Still I have mentioned the initial data wrangling steps that I would carry out if I am analyzing this data from scratch.

<b>Data Wrangling</b>

The given dataset has following issues that need to be fixed before we start working with the data:

1. There are 81 nan observations in Excess Readmissions column. We deleted them as they are small in number and should not affect our analysis.
2. There are some columns in data which are not needed for our analysis. So we deleted them.
3. There are multiple rows for same hospital/provider number. We combined the data to have a single row for each hospital. Provider number is the key. We summed each Predicted Readmission Rate and Expected Readmission Rate for each provider number. We then recalculated Excess Readmission Ratio by dividing Predicted Readmission Rate with Expected Readmission Rate.

In [ ]:
# Helper Functions

def pearson_r(x, y):
    """Compute Pearson correlation coefficient between two arrays."""
    # Compute correlation matrix: corr_mat
    corr_mat=np.corrcoef(x,y)

    # Return entry [0,1]
    return corr_mat[0,1]

In [ ]:
df=clean_hospital_read_df
df.info()

In [ ]:
df.shape

In [ ]:
np.sum(df['Excess Readmission Ratio'].isna())

In [ ]:
df['Provider Number'].value_counts().head()

In [ ]:
# Delete rows with Nan Excess Readmission Ratio
df = df.dropna(subset=['Excess Readmission Ratio'])

In [ ]:
# Drop unwanted columns

cols = [0,2,3,5,6,10,11]
df = df.drop(df.columns[cols],axis='columns',inplace = False).copy()

In [ ]:
# Combine rows with same Provider Number
df = df.groupby('Provider Number',as_index = False).sum()
df['Provider Number'].value_counts().head()

In [ ]:
# Calculate Excess Readmission Ratio and add it as a new column
df['Excess Readmission Ratio'] = round(df['Predicted Readmission Rate']/df['Expected Readmission Rate'],4)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
x = [a for a in df['Number of Discharges']]
y = list(df['Excess Readmission Ratio'])

fig, ax = plt.subplots(figsize=(9,5))
ax.scatter(x, y,alpha=0.2)

"""
ax.fill_between([0,1700], 1.15, 1.4, facecolor='red', alpha = .15, interpolate=True)
ax.fill_between([3500,212000], .5, .95, facecolor='green', alpha = .15, interpolate=True)
"""

ax.set_xlim([0, max(x)])
ax.set_xlabel('Number of discharges', fontsize=12)
ax.set_ylabel('Excess rate of readmissions', fontsize=12)
ax.set_title('Scatterplot of number of discharges vs. excess rate of readmissions', fontsize=14)

ax.grid(True)
fig.tight_layout()

In [ ]:
# Compute Pearson correlation coefficient: r
r = pearson_r(df['Number of Discharges'],df['Excess Readmission Ratio'])

# Print the result
print(r)

# Working on the given data (without performing the data wrangling steps)

In [ ]:
data = clean_hospital_read_df

# Drop rows with Nan values
data = data.dropna(subset=['Excess Readmission Ratio'])

In [ ]:
data.info()

In [ ]:
data.mean()

## My comments on the given analysis and recommendations

### A. Do you agree with the above analysis and recommendations? Why or why not?
   

**A. Initial observations based on the given scatter plot above** - Overall, rate of readmissions is trending down with increasing number of discharges

From the plot, it is not evident that rate of readmissions is negatively correlated with number of discharges. So, we calculate pearson coefficient of correlation.

In [ ]:
# Compute Pearson correlation coefficient: r
r = pearson_r(data['Number of Discharges'],data['Excess Readmission Ratio'])

# Print the result
print('r',r)

The rate of readmission is weakly correlated with number of discharge as evident from a small negative value, so the given observation that rate of readmissions is trending down with increasing number of discharges holds true. But to confirm that the negative correlation coefficient is not a chance occurence, we should conduct a hypothesis test.

**B. Statistics**
+ In hospitals/facilities with number of discharges < 100, mean excess readmission rate is 1.023 and 63% have excess readmission rate greater than 1 
+ In hospitals/facilities with number of discharges > 1000, mean excess readmission rate is 0.978 and 44% have excess readmission rate greater than 1 


In [ ]:
data_low_dischg=data[data['Number of Discharges']<100]

In [ ]:
data_low_dischg['Excess Readmission Ratio'].mean()

In [ ]:
np.sum(data_low_dischg['Excess Readmission Ratio']>1)/len(data_low_dischg)* 100

In [ ]:
data_high_dischg=data[data['Number of Discharges']>1000]

In [ ]:
data_high_dischg['Excess Readmission Ratio'].mean()

In [ ]:
np.sum(data_high_dischg['Excess Readmission Ratio']>1)/len(data_high_dischg)*100

The given statistical findings are verified by above calculations. But these numbers would be completely different if we carry out these calculations after wrangling the data as explained above.

The calculations with wrangled data are below:

In [ ]:
d_l=df[df['Number of Discharges']<100]

In [ ]:
d_l['Excess Readmission Ratio'].mean()

In [ ]:
np.sum(d_l['Excess Readmission Ratio']>1)/len(d_l)* 100

In [ ]:
d_h=df[df['Number of Discharges']>1000]

In [ ]:
d_h['Excess Readmission Ratio'].mean()

In [ ]:
np.sum(d_h['Excess Readmission Ratio']>1)/len(d_h)*100

On the basis of calculations on the wrangled data, we can modify the findings as follows:

+ In hospitals/facilities with number of discharges < 100, mean excess readmission rate is 1.018 and 62% have excess readmission rate greater than 1 
+ In hospitals/facilities with number of discharges > 1000, mean excess readmission rate is 1.002 and 52% have excess readmission rate greater than 1 

## B. Provide support for your arguments and your own recommendations with a statistically sound analysis:

### 1. Setup an appropriate hypothesis test.



### Null Hypothesis
There is no correlation between number of discharges and readmission rates.

### Alternate Hypothesis
There is significant correlation between number of discharges and readmission rates.

### 2. Compute and report the observed significance value (or p-value).


In [ ]:
data.head()

In [ ]:
r_obs = pearson_r(data['Number of Discharges'],data['Excess Readmission Ratio'])
print('r_obs=',r_obs)

# Initialize permutation replicates: perm_replicates
perm_replicates = np.empty(10)

# Draw replicates
for i in range(10):
    # Permute illiteracy measurments: illiteracy_permuted
    discharge_permuted = np.random.permutation(data['Number of Discharges'].copy())

    # Compute Pearson correlation
    perm_replicates[i] = pearson_r(discharge_permuted,data['Excess Readmission Ratio'])

# Compute p-value: p
p = np.sum(perm_replicates<=r_obs)/len(perm_replicates)
print('p-val =', p)

###    3. Report statistical significance for $\alpha$ = .01. 


For α  = .01, as p-value is less than α, we reject the null hypothesis and conclude that  number of discharges and readmission rates are negatively correlated.

In [ ]:
stats.pearsonr(data['Number of Discharges'],data['Excess Readmission Ratio'])

### 4. Discuss statistical significance and practical significance. Do they differ here? How does this change your recommendation to the client?

The hypothesis test conducted above shows that there is significant correlation between  number of discharges and readmission rates. However, practically speaking, as the value of r is close to zero, we cannot say that number of discharges and readmission rates are strongly correlated. It shows a weak correlation. So statistical significance is different from practical significance.

So practically speaking, as number of discharges and readmission rates seem to be weakly correlated, I would not recommend the client to take any actions as given in the recommendations above.

### 5. Look at the scatterplot above. 
      - What are the advantages and disadvantages of using this plot to convey information?
      - Construct another plot that conveys the same information in a more direct manner.


The scatter plot above does not clearly show the relationship between discharges and admission rates. Adding a regression line to the plot would be better.

In [ ]:
sns.jointplot('Number of Discharges', 'Excess Readmission Ratio', data=data,
              kind='reg')
plt.show()